## Dynamic visualization of California fires in 2014

GOAL: Creating a plot with dynamic visualizations of daily wildfires burning in CA


In [1]:
import time
import pandas as pd
import numpy as np
import sqlite3
import os
from datetime import date, timedelta
import datetime
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import shapefile
import geopandas as gpd
from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *
from scripts import *

In [2]:
# establishing db connection

con = sqlite3.connect("FPA_FOD_20170508.sqlite")

In [3]:
# reading data from database
cali_fires = pd.read_sql_query("""SELECT STATE, FIRE_SIZE_CLASS,FIRE_SIZE, LONGITUDE, LATITUDE, FIRE_YEAR,
date(DISCOVERY_DATE) as DISCOVERY_DATE,date(CONT_DATE) as CONT_DATE
FROM Fires
WHERE STATE = "CA" and FIRE_YEAR = 2014
""", con)

In [4]:
# translating fire size class to numerical values
cali_fires.FIRE_SIZE_CLASS = cali_fires.FIRE_SIZE_CLASS.map({"A":1,"B":4,"C":8,"D":12,"E":15,"F":20,"G":30})

In [5]:
# creating geodataframe, resizing fire size class
cali_gdf = geodf_create_transform(cali_fires,"LONGITUDE","LATITUDE",to_resize="FIRE_SIZE_CLASS",resize=0.001)

C:\Users\szklarnia\Desktop\datascience\projekty\us wildfires\scripts.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.x_crds[i] = list(gdf.geometry[i].coords)[0][0]
C:\Users\szklarnia\Desktop\datascience\projekty\us wildfires\scripts.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.y_crds[i] = list(gdf.geometry[i].coords)[0][1]


In [6]:
# dropping geometry column 
cali_gdf.drop("geometry",axis=1,inplace=True)

In [7]:
# sepcyfying map range
cali_max_x, cali_max_y = get_max_min(-111,-128 ,32 ,43)

In [8]:
# translating dates to datetime format
cali_gdf.DISCOVERY_DATE = [date.fromisoformat(x) for x in cali_gdf.DISCOVERY_DATE]
cali_gdf.CONT_DATE = cali_gdf.CONT_DATE.fillna("2014-12-31")
cali_gdf.CONT_DATE = [date.fromisoformat(x) for x in cali_gdf.CONT_DATE]

In [9]:
# computing lenght of fires
days_between = []
for i in range(len(cali_gdf.CONT_DATE)):
    days_between.append([cali_gdf.DISCOVERY_DATE[i] + timedelta(days=x) for x in range((cali_gdf.CONT_DATE[i]-cali_gdf.DISCOVERY_DATE[i]).days + 1)])

In [10]:
# creating data frame with fires burning on each day of year
fires_burning_daily = pd.DataFrame()

for i in range(len(days_between)):
    for day in days_between[i]:
        series_day = cali_gdf.iloc[i]
        series_day["day"] = day
        fires_burning_daily = fires_burning_daily.append(series_day,ignore_index=True)

<ipython-input-10-cd9a11384b1d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series_day["day"] = day
C:\Users\szklarnia\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [11]:
# creating geodataframe
daily_gdf = gpd.GeoDataFrame(fires_burning_daily)

In [14]:
# plotting dynamic plot
# bokeh source
source_cali = ColumnDataSource(daily_gdf[daily_gdf.day == datetime.date(2014,1,1)])

# amount of days in a ear
days = [datetime.date(2014,1,1) + timedelta(days=x) for x in range((datetime.date(2014,12,31)-datetime.date(2014,1,1)).days + 1)]

# plotting bubble map
p = plot_bubble_map_for_animation(df=daily_gdf[daily_gdf.day == datetime.date(2014,1,1)],  
               source=source_cali,
               radius_col="resized",
               hover_tuples=[("Size [in acres]:","@FIRE_SIZE")],
               x_range=cali_max_x,
               y_range=cali_max_y,
               title = f"Wildfiręs in CA on {datetime.date(2014,1,1)} ",
               leg_label = "Wildfires by size")

# creating handle
handle = show(p, notebook_handle = True)

# updating map daily
for day in days:
    # df for day 
    df = daily_gdf[daily_gdf.day == day]
    # changing title
    p.title.text = f"Wildfires in california on {day}"
    # Push new data
    source_cali.stream(df)
    # Purge old data
    source_cali.data = df
    push_notebook(handle=handle)
    time.sleep(.1)